In [26]:
import pandas as pd
from statsmodels.imputation import mice
import numpy as np
from sklearn.model_selection import cross_val_score

# import submission function
from data_munge import make_country_sub, impute

In [2]:
# import the raw training data for reference
from data_munge import a_i_train, a_h_train, b_i_train, b_h_train, c_i_train, c_h_train

# import the preprocessed test data
from data_munge import a_i_test, a_h_test, b_i_test, b_h_test, c_i_test, c_h_test

# import the preprocessed data
from data_munge import aX_h_train, ay_h_train, bX_h_train, by_h_train, cX_h_train, cy_h_train
from data_munge import aX_i_train, ay_i_train, bX_i_train, by_i_train, cX_i_train, cy_i_train 

In [3]:
bX_h_train = impute(bX_h_train)
b_h_test_imputed = impute(b_h_test)

In [8]:
from sklearn import svm

def train_model(features, labels, **kwargs):
    
    # instantiate model
    model = svm.SVC(probability=True)
    
    # train model
    model.fit(features, labels)
    
    # get a (not-very-useful) sense of performance
    accuracy = model.score(features, labels)
    print(f"In-sample accuracy: {accuracy:0.2%}")
    
    return model

In [9]:
model_a = train_model(aX_h_train, ay_h_train)

In-sample accuracy: 87.96%


In [10]:
model_b = train_model(bX_h_train, by_h_train)

In-sample accuracy: 92.29%


In [11]:
model_c = train_model(cX_h_train, cy_h_train)

In-sample accuracy: 90.52%


In [12]:
a_preds = model_a.predict_proba(a_h_test)
b_preds = model_b.predict_proba(b_h_test_imputed)
c_preds = model_c.predict_proba(c_h_test)

In [13]:
a_sub = make_country_sub(a_preds, a_h_test, 'A')
b_sub = make_country_sub(b_preds, b_h_test_imputed, 'B')
c_sub = make_country_sub(c_preds, c_h_test, 'C')

In [14]:
submission = pd.concat([a_sub, b_sub, c_sub])

In [15]:
submission.to_csv('submissions/svm_1.csv')

In [16]:
submission_old = pd.read_csv('submissions/imputation_household_2.csv', index_col='id')

In [24]:
np.mean(submission_old.poor - submission.poor)

0.2553659800309075

In [29]:
scores = cross_val_score(model_a, aX_h_train, ay_h_train, cv=5, n_jobs=-1)

In [30]:
scores

array([ 0.8622791 ,  0.86837294,  0.86166971,  0.87621951,  0.8597561 ])